# Example: CitiBike data

Adapted from Kelsey Jordahl
https://gist.github.com/kjordahl/5957573

In [2]:
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp
import folium
import pandas as pd

NYC borough boundaries downloaded from [Bytes of the Big Apple](http://www.nyc.gov/html/dcp/download/bytes/nybb_13a.zip)

In [3]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')

Load real time bike station data from [CitiBike](http://citibikenyc.com) json API:

In [4]:
# get the bike data and convert it to a ditcionary
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)


In [5]:
# convert the relevant part to a geodataframe
df = gp.GeoDataFrame(data['stationBeanList'])
df.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,14,24,,72,,2017-03-10 04:11:55 PM,40.767272,,-73.993929,,W 52 St & 11 Ave,,W 52 St & 11 Ave,1,In Service,False,39
1,,15,18,,79,,2017-03-10 04:12:52 PM,40.719116,,-74.006667,,Franklin St & W Broadway,,Franklin St & W Broadway,1,In Service,False,33
2,,12,15,,82,,2017-03-10 04:09:38 PM,40.711174,,-74.000165,,St James Pl & Pearl St,,St James Pl & Pearl St,1,In Service,False,27
3,,28,34,,83,,2017-03-10 04:09:21 PM,40.683826,,-73.976323,,Atlantic Ave & Fort Greene Pl,,Atlantic Ave & Fort Greene Pl,1,In Service,False,62
4,,4,35,,116,,2017-03-10 04:10:57 PM,40.741776,,-74.001497,,W 17 St & 8 Ave,,W 17 St & 8 Ave,1,In Service,False,39


In [6]:
# there is one row for each bike station.  How many stations are there? 
len(df)

665

In [7]:
# in the file above, there are lon-lats, but no geometry field
# we need to set that up

s = gp.GeoSeries([Point(x, y) for x, y in zip(df['longitude'], df['latitude'])])
df['geometry'] = s
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df.geometry.total_bounds

(-74.096936600000006,
 40.661063371900603,
 -73.929891100000006,
 40.804212999999997)

In [8]:
# make sure they are on the same CRS.  
# checking the bounds is a nice way of seeing this
df.to_crs(boros.crs, inplace=True)
df.geometry.total_bounds

(957370.14732175611,
 180120.27032614074,
 1003695.8507545569,
 232275.23054640222)

In [9]:
# the geometry objects can do lots of cool stuff.  For example: 

manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'stations in Manhattan')

366 stations in Manhattan


# Your turn

You can read about the range of operations available in geopandas here: 
    
http://geopandas.org/index.html

Your assignment is to: 

1. Calculate how many stations are in each borough
2. Calculate how many bikes are currently available in each borough
3. Read about and try at least two new spatial or geometric operations (beyond what I've covered here). 

In [10]:
#Function to create boolean columns in df dataframe
def bike_in_boro(x):
    boro = boros.geometry[x]
    in_boro = df.geometry.within(boro)
    return in_boro

In [11]:
#Creates columns
for j in range(0,5):
    if j == 0:
        b = bike_in_boro(j)
        df['Statisland']=b
    elif j == 1:
        b = bike_in_boro(j)
        df['Brooklyn']=b
    elif j == 2:
        b = bike_in_boro(j)
        df['Queens']=b
    elif j == 3:
        b = bike_in_boro(j)
        df['Manhattan']=b
    elif j == 4:
        b = bike_in_boro(j)
        df['Bronx']=b

In [12]:
#Creates a new bikes dataframe to carry bike values:
bikes = pd.DataFrame(index = range(len(df)),columns = ['Statisland','Brooklyn','Queens','Manhattan','Bronx'])
bikes.head()

,Statisland,Brooklyn,Queens,Manhattan,Bronx
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


In [13]:
bikes.iloc[0,0]

nan

In [17]:
df.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,...,statusKey,statusValue,testStation,totalDocks,geometry,Statisland,Brooklyn,Queens,Manhattan,Bronx
0,,14,24,,72,,2017-03-10 04:11:55 PM,40.767272,,-73.993929,...,1,In Service,False,39,POINT (985931.706502895 218814.7571059909),False,False,False,True,False
1,,15,18,,79,,2017-03-10 04:12:52 PM,40.719116,,-74.006667,...,1,In Service,False,33,POINT (982402.0068864136 201269.770346975),False,False,False,True,False
2,,12,15,,82,,2017-03-10 04:09:38 PM,40.711174,,-74.000165,...,1,In Service,False,27,POINT (984204.131576321 198376.4207269395),False,False,False,True,False
3,,28,34,,83,,2017-03-10 04:09:21 PM,40.683826,,-73.976323,...,1,In Service,False,62,POINT (990816.693171227 188413.5778061197),False,True,False,False,False
4,,4,35,,116,,2017-03-10 04:10:57 PM,40.741776,,-74.001497,...,1,In Service,False,39,POINT (983835.0432398689 209525.6347344065),False,False,False,True,False


In [14]:
len(bikes)

665

In [18]:
#What Works. Also, df is the same dataframe that was created in the citibike tutorial. It has already had rows added to where if the bike location is within a given boro, it is true and if not it is false.
if df.iloc[0,19] == True:
    bikes.iloc[0,0] = df.iloc[0,1]
else:
    bikes.iloc[0,0] = [0]

In [19]:
bikes.iloc[0,0]

[0]

In [20]:
#My Function
def bike_index(r,k):
    if df.iloc[r,k+19] == True:
        bikes.iloc[r,k] = df.iloc[r,1]
    else:
        bikes.iloc[r,k] = 0

In [21]:
#What I want to do
for k in range (0,5):
    for x in range (0,len(bikes)):
        bike_index(x,k)

In [22]:
bikes

,Statisland,Brooklyn,Queens,Manhattan,Bronx
0,0,0,0,14,0
1,0,0,0,15,0
2,0,0,0,12,0
3,0,28,0,0,0
4,0,0,0,4,0
5,0,13,0,0,0
6,0,2,0,0,0
7,0,0,0,25,0
8,0,0,0,10,0
9,0,0,0,0,0


In [23]:
bikes.sum()

Statisland       0.0
Brooklyn      2917.0
Queens         175.0
Manhattan     4418.0
Bronx            0.0
dtype: float64